In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable
from langchain_ollama import ChatOllama

large_model = ChatOllama(model="llama3.1:8b", temperature=0)
standard_model = ChatOllama(model="mistral:7b", temperature=0)


@wrap_model_call
def state_based_model(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """Select model based on State conversation length."""
    # request.messages is a shortcut for request.state["messages"]
    message_count = len(request.messages)  

    if message_count > 10:
        # Long conversation - use model with larger context window
        model = large_model
    else:
        # Short conversation - use efficient model
        model = standard_model

    request = request.override(model=model)  

    return handler(request)

In [3]:
from langchain.agents import create_agent

agent = create_agent(
    model=large_model,
    middleware=[state_based_model],
    system_prompt="You are roleplaying a real life helpful office intern."
)

In [4]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?")
        ]}
)

print(response["messages"][-1].content)

 I'm an AI and don't have physical capabilities to perform tasks like watering plants. However, if there's an office plant that needs watering, I can remind the team about it! Let me check if anyone has taken care of it today. If not, I'll make sure to send a friendly reminder to keep our green friend healthy and thriving.


In [5]:
print(response["messages"][-1].response_metadata["model_name"])

mistral:7b


In [6]:
from langchain.messages import AIMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?"),
        AIMessage(content="Yes, I gave it a light watering this morning."),
        HumanMessage(content="Has it grown much this week?"),
        AIMessage(content="It's sprouted two new leaves since Monday."),
        HumanMessage(content="Are the leaves still turning yellow on the edges?"),
        AIMessage(content="A little, but it's looking healthier overall."),
        HumanMessage(content="Did you remember to rotate the pot toward the window?"),
        AIMessage(content="I rotated it a quarter turn so it gets more even light."),
        HumanMessage(content="How often should we be fertilizing this plant?"),
        AIMessage(content="About once every two weeks with a diluted liquid fertilizer."),
        HumanMessage(content="When should we expect to have to replace the pot?")
        ]}
)

print(response["messages"][-1].content)

I'd say in about 6-8 months, it's been growing pretty steadily and will likely need a bigger pot by then. I can start looking into options for a new one if you'd like.


In [7]:
print(response["messages"][-1].response_metadata["model_name"])

llama3.1:8b
